In [13]:
%load_ext autoreload
%autoreload 2

In [29]:
from mlp import PredictorMLP
from mlprunner import train_mlp
from config import GameConfig
from player import load_players
from enums import StartingSplit, FeatureType
from predictor import sample_training_batch, test_mlp
import numpy as np
import torch.nn as nn
import torch
import os
import pandas as pd
from sklearn.metrics import mean_squared_error

In [30]:
config = GameConfig(split=StartingSplit.StartingRandomSplit, prosperity=False, num_players=2, feature_type=FeatureType.ReducedFeature, sandbox=True)
players = load_players(['BM', 'BM'], models=None, train=True)

In [35]:
X, y = sample_training_batch(1000, 10, config, players)

100%|██████████| 1000/1000 [00:06<00:00, 146.53it/s]


In [31]:
model = PredictorMLP(config.feature_size, (config.feature_size + 1) // 2, 1)
criterion = nn.MSELoss()

In [32]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [33]:
model_name = 'bm-bm-mlp-pred-2k-50-10'

In [36]:
y1 = np.array(y, dtype=np.float32)
train_mlp(X, y1, model, nn.BCELoss(), epochs=50, save_epochs=10, model_name=model_name, path=os.path.join(model_dir, model_name))

100%|██████████| 50/50 [11:57<00:00, 14.35s/it]


In [65]:
y_pred = model(torch.tensor(X).cuda())
y_labels = np.array(y).reshape(y_pred.shape)

In [87]:
mean_squared_error(y_pred, y_labels)

0.11117749924175713